In [1]:
import numpy as np
import pandas as pd
arrSries  = pd.Series()
fns = ['2HTaSe2_bad_78K097','2HTaSe2_ap_118K002','2HTaSe2_bae_110K012','2HTaSe2_ao_115K037','test']

for fn in fns:
    arrSries[fn] = np.load(f'{fn}_cln.npy')

In [2]:
from useful import *
tab = subtabSr(arrSries)

d:\github\2H_TaSe2_Tc_STM\2Hvenv\Lib\site-packages\numpy\lib\_function_base_impl.py:4968: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


In [3]:
tab

In [4]:
from scipy.fft import fft2, fftshift
from skimage.feature import peak_local_max
def fft2show(arr_cln,vmin,vmax):
  fft_result = fft2(arr_cln)
  fft_result_shifted = fftshift(fft_result)
  magnitude_spectrum = np.abs(fft_result_shifted)
  plt.figure(figsize=(10, 10))
  plt.imshow(magnitude_spectrum, cmap='gray', vmin=vmin, vmax=vmax)
  plt.title('Magnitude Spectrum of 2D FFT')
  return  magnitude_spectrum
def fft2pkfnd(fft2abs,threshold,choose):
  coordinates = peak_local_max(fft2abs, min_distance=100, threshold_abs = threshold)
  plt.scatter(coordinates[:, 1], coordinates[:, 0], s=50, facecolors='none', edgecolors='r')
  for ipeak in range(coordinates.shape[0]):
    plt.text(coordinates[ipeak,1],coordinates[ipeak,0],str(ipeak), color='g')
  coordinates_choose = coordinates[choose,:]
  print(coordinates_choose)
  print(coordinates_choose.shape[0])
  for ipeak in range(coordinates_choose.shape[0]):
    plt.text(coordinates_choose[ipeak,1],coordinates_choose[ipeak,0],str(ipeak), color='b',size= 20)
    pk_all = coordinates - (np.array(fft2abs.shape)/2)
    pk_choose = pk_all[choose,:]
  return pk_choose,pk_all

In [5]:
vmin = 0 # @param
vmax = 0.0000001 # @param
threshold = 0#0.0000000001 # @param
choose = [0] # @param

In [6]:
from matplotlib import pyplot as plt

In [7]:
chooses = [
    [7,8,11],
    [1,4,3],
    [1,4,7],
    [1,2,5],
    [7,2,5]
]

In [8]:
choose = chooses[0]

In [9]:
ffts = []
for fn,choose in zip(fns,chooses):
    arr_cln = arrSries[fn]
    fft2abs = fft2show(arr_cln,vmin,vmax)
    pk_choose,pk_all = fft2pkfnd(fft2abs,threshold,choose)
    ffts.append(plt.gcf()) 
    plt.close()
        

[[1138 1547]
 [ 494  847]
 [1450  679]]
3
[[274 398]
 [111 208]
 [382 162]]
3
[[559 793]
 [216 417]
 [763 327]]
3
[[543 654]
 [363 464]
 [630 418]]
3
[[1076 1282]
 [ 747  937]
 [1249  854]]
3


In [10]:
subtabs(ffts,fns)

In [11]:
import numpy as np

def autocorr2d_fft(arr):
    f = np.fft.fft2(arr)
    psd = f * np.conj(f)   # power spectral density
    result = np.fft.ifft2(psd).real
    return np.fft.fftshift(result)  # 중앙 정렬

arr = np.random.rand(5, 5)
auto_corr_fft = autocorr2d_fft(arr)

print(auto_corr_fft.shape)

(5, 5)


In [12]:
cntr = np.array(arr.shape)/2

In [13]:
auto_corr_ffts = []
auto_corr_fft_figs= []
for ind in range(len(fns)):
    fn = fns[ind]
    arr = arrSries[fn]
    auto_corr_fft = autocorr2d_fft(arr)
    auto_corr_ffts.append(auto_corr_fft)
    fig = plt.figure(figsize=(10,10))
    plt.imshow(auto_corr_fft, cmap='gray')
    auto_corr_fft_figs.append(fig) 
    tickoff()
    cntr = np.array(arr.shape)/2
    plt.xlim(cntr[1]-50,cntr[1]+50)
    plt.ylim(cntr[0]-50,cntr[0]+50)
    plt.close()
    

In [14]:
subtabs(auto_corr_fft_figs,fns)

In [15]:
from scipy.ndimage import map_coordinates

def line_profile(image, p1, p2):
    """
    Get the line profile of a 2D image between two points p1 and p2.
    
    Args:
        image (np.ndarray): The 2D image.
        p1 (tuple): The (y, x) coordinates of the start point.
        p2 (tuple): The (y, x) coordinates of the end point.
        
    Returns:
        np.ndarray: The 1D array of intensity values along the line.
    """
    y1, x1 = p1
    y2, x2 = p2
    
    # Number of points in the line profile
    num_points = int(np.ceil(np.linalg.norm(np.array(p1) - np.array(p2))))
    
    # Generate coordinates along the line
    y_coords = np.linspace(y1, y2, num_points)
    x_coords = np.linspace(x1, x2, num_points)
    
    # `map_coordinates` expects coordinates in (ndim, n_points) format
    coords = np.vstack((y_coords, x_coords))
    
    # Interpolate the values from the image
    profile = map_coordinates(image, coords, order=1)
    
    return profile

# Example usage:
# Define two points for the line profile
# p1 = (1024, 1024)  # Center of the image




In [122]:
p2s =[
    [1038,962],
    [263,200],
    [521,457],
    [537,401],
    [1048,905]
    ]

In [128]:
idx =3
auto_corr_fft = auto_corr_ffts[idx]
p1 = np.array(auto_corr_fft.shape)/2
# p2 = (1024 + pk_choose[0, 0], 1024 + pk_choose[0, 1]) # A chosen peak
p2 = p2s[idx][1],p2s[idx][0]
# crp = 400

# Get the line profile from the auto-correlation image
profile = line_profile(auto_corr_fft, p1, p2)

# Plot the result
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plot the image with the line
axes[0].imshow(auto_corr_fft, cmap='gray')
axes[0].plot([p1[1], p2[1]], [p1[0], p2[0]], 'ro') # Draw line
axes[0].set_title('Image with Line Profile')
axes[0].axis('image')
cntr = p1
# axes[0].set_xlim(cntr[1]-crp,cntr[1]+crp)
# axes[0].set_ylim(cntr[0]-crp,cntr[0]+crp)

# Plot the line profile
axes[1].plot(profile)
axes[1].set_title('Line Profile')
axes[1].set_xlabel('Distance along line (pixels)')
axes[1].set_ylabel('Intensity')
axes[1].grid(True)

plt.show(block=True)

In [66]:
from scipy.ndimage import map_coordinates

def line_profile(image, p1, p2,num_points=1000):
    """
    Get the line profile of a 2D image between two points p1 and p2.
    
    Args:
        image (np.ndarray): The 2D image.
        p1 (tuple): The (y, x) coordinates of the start point.
        p2 (tuple): The (y, x) coordinates of the end point.
        
    Returns:
        np.ndarray: The 1D array of intensity values along the line.
    """
    y1, x1 = p1
    y2, x2 = p2
    
    # Number of points in the line profile
    # num_points = int(np.ceil(np.linalg.norm(np.array(p1) - np.array(p2))))
    
    # Generate coordinates along the line
    y_coords = np.linspace(y1, y2, num_points)
    x_coords = np.linspace(x1, x2, num_points)
    
    # `map_coordinates` expects coordinates in (ndim, n_points) format
    coords = np.vstack((y_coords, x_coords))
    
    # Interpolate the values from the image
    profile = map_coordinates(image, coords, order=1)
    
    return profile

In [71]:
idt = 0
auto_corr_fft = auto_corr_ffts[idt]
p1 = np.array(auto_corr_fft.shape)/2
p2 = p2s[idt]
lp = line_profile(auto_corr_fft, p1, p2,num_points=1000)
plt.plot(lp)

In [ ]:
idt = 0
auto_corr_fft = auto_corr_ffts[idt]
p1 = np.array(auto_corr_fft.shape)/2
p2 = p2s[idt]
p1 = 2*p1 - p2
from skimage.measure import profile_line

# Using auto_corr_fft, p1, and p2 from the previous cells
# The function expects (row, col) coordinates, which is the format of p1 and p2.
profile_sk = profile_line(auto_corr_fft, p1, p2,linewidth=5)

# Plot the resulting line profile
plt.figure(figsize=(12, 6))
plt.plot(profile_sk)
plt.title('Line Profile using skimage.measure.profile_line')
plt.xlabel('Distance along line (pixels)')
plt.ylabel('Intensity')
plt.grid(True)
plt.show()

In [133]:
lps = []
for idt in range(1,5):
    auto_corr_fft = auto_corr_ffts[idt]
    p1 = np.array(auto_corr_fft.shape)/2
    p2 = p2s[idt][1],p2s[idt][0]
    # p1 = 2*p1 - p2
    lp = line_profile(auto_corr_fft, p1, p2,num_points=1000)
    # lp = profile_line(auto_corr_fft, p1, p2,linewidth=5)
    lp = lp/np.max(lp)
    lps.append(lp)
    plt.plot(lp,label=fns[idt],linewidth=2)
# plt.vertical_line(0)
plt.legend()    
plt.axhline(0,color='k')

In [129]:
type(p2)

tuple